# Lecture 11: Inheritance part 2

## Reading
*  Composing Programs: Section 2.5.5-2.5.8
    * https://www.composingprograms.com/pages/25-object-oriented-programming.html 
* Chapter 10 of Guttag
* Chapter 30 of Lutz

# Object-oriented programming
* In OOP, all the data for a program is grouped into various different objects.
* The objects have an interface that defines how they interact with one another.
* The interactions and operations are performed mostly by objects via methods that may or may not affect an object (or another object's) state (i.e. its data).
* Classes can be implemented and tested independently.
* The concept of <b> inheritance</b> allows a subclass to redefine or extend it's superclass' behavior.
    * Again, this helps with code reduction and complexity. 


## Case study
Let's convert Lab 4 - The 21 Card Game into an object-oriented program.

First, think of what kind of objects we need

- Card
    * Previously, we defined cards as a tuple (a built in type) with a value and a suit 
    ```python

    vals = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
    suits = ['spades', 'clubs', 'hearts', 'diamonds']
    ```
- A card deck
    * Just a collection of cards. Previously we represented this as a set
    ```python
    deck = set([(v, s) for v in vals for s in suits])
    ```
    
- A player
    * Player's hold cards
    * Player might also be a dealer
        - A more specific version of a player

We will use `random` to shuffle the Cards so import random

In [1]:
import random

## Card class

| Attribute| Class or Instance|Description|
--------------|-------|---------------
| `SUITS `| <b> Class  </b> | Set of all card suits.|
| `VALS `| <b> Class  </b> | Set of all card values.|
| `suit `| <b> Instance  </b>|  Card object's suit.|
| `val `| <b> Instance  </b> | Card object's value.|


| Method| 	Description|
--------------|----------------------
| `__init__() `| 	Constructor|
|`__str__()`|	Returns a string representation of the Card.|

In [2]:
class Card:
    '''Represents a card object'''
    
    SUITS = {'spades', 'clubs', 'hearts', 'diamonds'}
    VALS = {'2', '3', '4', '5', '6', '7', '8', '9', 
            '10', 'queen', 'king', 'jack', 'ace'}
    
    def __init__(self, val, suit):
        '''Creates a card with the given suit and value.'''
        self.suit = suit
        self.val = str(val)
        
    def __str__(self):
        '''Returns the string representation of a card. '''
        return self.val + ' of ' + self.suit

In [5]:
c = Card(4, 'clubs')
print(c)
print(c.suit)
print(c.val)

4 of clubs
clubs
4


## Deck class

| Attribute| Class or Instance|Description|
--------------|-------|---------------
| `hand `| <b> Instance  </b> |List of cards in the deck.|


| Method| 	Description.|
--------------|----------------------
| `__init__()` | 	Constructor.|
| `__len__()`| Returns the number of cards currently in the deck. Called with len(d)| 
| `shuffle()`| Shuffles the cards in the deck.| 
|`deal()` | If the deck is not empty, removes and returns the topmost card. Otherwise, returns None . |
|`__str__()`|	Returns a string representation of the deck (all the cards in it). Called with str(d).|

In [6]:

class Deck:
    '''A deck containing 52 cards.'''
    
    def __init__(self):
        '''Creates a full deck of cards.'''
        self.hand = []
    
        for suit in Card.SUITS:
            for val in Card.VALS:
                c = Card(val, suit) 
                self.hand.append(c)
            
    def shuffle(self):
        '''Shuffles the cards.'''
        random.shuffle(self.hand)
       
    def deal(self):
        '''Removes and returns the top n cards  
         or None if the deck is empty.'''
        
        if len(self.hand) > 0:
            self.shuffle()
            return self.hand.pop(0)
        
    def __len__(self):
        '''Returns the number of cards left in the deck.'''
        return len(self.hand)
    
    def __str__(self):
        '''Returns the string representation of a deck.'''
        result = ''
        for card in self.hand: 
            result = result + str(card) + '\n' 
        return result

In [10]:
deck =  Deck()

In [12]:
print(len(deck))

52


## Player Class

| Attribute| Class or Instance|Description|
--------------|-------|---------------
| `hand `| <b> Instance  </b> |List of cards in the player's hand.|
| `score `| <b> Instance  </b> |Best score of player's hand.|


| Method| 	Description.|
--------------|----------------------
| `__init__()` | 	Constructor. Initializes hand and calculates best score of the hand.|
|`__str__()`|	Returns a string representation of the player|
| `draw_card()`| Takes a Deck object as input and draws a random card using `Deck.deal()`| 
| `calc_score()`| Calculates the best score of `self.hand.`| 


In [13]:
class Player: 
    ''''This class represents a player in a game of 21.'''
    
    def __init__(self, hand=None):
        self.hand = hand
        self.calc_score()

    def __str__(self): 
        '''Returns string rep of hand and points.'''
        rtn = '\tCards: '
        rtn += ', '.join(map (str, self.hand)) 
        rtn += '\n\tBest score: ' + str(self.score)
        return rtn

    def draw_card(self, deck):
        '''Draw a card from the deck'''
        self.hand.append(deck.deal())
        self.calc_score()

    def calc_score(self): 
        '''Returns the best score given the cards in the Player's hand. '''

        # This function is almost identical to self.score() in Lab 4
        score_without_aces = 0
        number_of_aces = 0

        # iterate through each card in the set
        for card in self.hand:
            v = card.val
            s = card.suit

            # check if its an ace
            if v == 'ace':
                number_of_aces = number_of_aces + 1
            else:
                # if not, get its value
                if v in {'king', 'queen', 'jack'}:
                    score_without_aces = score_without_aces + 10
                else:
                    score_without_aces = score_without_aces + int(v)   

        # now deal with the aces (if any) and find the best score
        if number_of_aces == 0:
            self.score = score_without_aces

        # 1 ace, either add 10 or 1
        elif number_of_aces == 1:
            if score_without_aces + 10 <= 21:
                self.score = score_without_aces + 10
            else:
                self.score = score_without_aces + 1

        # 2 aces, we either add 20, 11, or 2
        elif number_of_aces == 2:
            if score_without_aces + 20 <= 21:
                self.score = score_without_aces + 20
            elif  score_without_aces + 11 <= 21:
                self.score = score_without_aces + 11
            else:
                self.score = score_without_aces + 2

        # 3 aces, either add 21, 12, or 3
        elif number_of_aces == 3: 
            if score_without_aces + 21 <= 21:
                self.score = score_without_aces + 21
            elif  score_without_aces + 12 <= 21:
                self.score = score_without_aces + 12
            else:
                self.score = score_without_aces + 3

        # 4 aces we can only have 13 or 4
        else: 
            if score_without_aces + 13 <= 21:
                self.score = score_without_aces + 10 + 3
            else:
                self.score = score_without_aces + 4


In [15]:
customer = Player([])
print(customer)

	Cards: 
	Best score: 0


In [ ]:
print(len(deck))

In [ ]:
customer.draw_card(deck)

In [ ]:
print(len(deck))

In [19]:
print(customer)

	Cards: 10 of spades
	Best score: 10


In [21]:
customer.draw_card(deck)
customer.draw_card(deck)
print(customer)

	Cards: 10 of spades, 10 of hearts, 2 of spades, jack of diamonds, ace of spades
	Best score: 33


In [22]:
print(len(deck))

47


## Dealer
A dealer is a more specific player. The Dealer has two main differences between a general Player:
    
* The dealer has a visible card -- this is an additional instance variable.
    * Because the dealer has to have a visible card, we initialize the Dealer with at least one card drawn.
* The dealer uses a `stop_at_seventeen` strategy -- this will be incorporated in the Dealer's `draw_card()` method -- instead of drawing a card like all other Players, the Dealer will make sure to stop when their score is 17. So, we will re-define `draw_card()`.
    

| Attribute| Class or Instance|Description|
--------------|-------|---------------
| `visible_card `| <b> Instance  </b> |A Card object that remains visible to the other Players|


| Method| 	Description.|
--------------|----------------------
| `__init__()` | 	Constructor. Includes the initialization of `self.visible_card`.|
|`__str__()`|	Returns a string representation of the Dealer.|
| `draw_card()`| Takes a Deck object as input and draws a random card using `Deck.deal()`| 


In [23]:
class Dealer(Player):
    """Represents a Dealer in a game of 21. An extension of a Player"""
    
    def __init__(self, hand):
        """Initiates a Dealer object. """
        Player.__init__(self, hand)
        self.visible_card = self.hand[0]
        
    def __str__(self):
        """Returns string representation of Dealer (only the visible card)"""

        return "Visible card: " + str(self.visible_card)
    
    def draw_card(self, deck):
        """ Draw a card from a deck only if self.score < 17"""
        if self.score < 17:
            c = deck.deal()
            self.hand.append(c)

In [24]:
dealer = Dealer([deck.deal()])

In [25]:
print(dealer)

Visible card: queen of diamonds


In [26]:
print(len(deck))

46


## The actual Game
To play the actual game, create all the objects we need, and ask the objects to perform their actions.

Objects needed:
* A deck of cards
* A dealer
* A player

### Game class

| Attribute| Class or Instance|Description|
--------------|-------|---------------
| `deck `| <b> Instance  </b> |The deck of cards (Deck object)|
| `dealer `| <b> Instance  </b> |Dealer object|
| `customer `| <b> Instance  </b> |Player object -- the customer|


| Method| 	Description.|
--------------|----------------------
| `__init__()` | 	Constructor. Create the Deck. Create the Dealer. Create the Player. |
| `play()`| Play the game. | 


In [28]:
class TwentyOne:
    ''' A game of twenty one.
    Consists of a Deck, a Player and a Dealer.
    '''
    def __init__(self):
        '''Initialize the game.
        Create the Deck, create the dealer, create the customer
        Both the dealer and customer start off with two cards
        '''
        self.deck = Deck()
        self.deck.shuffle()

        self.dealer =  Dealer([self.deck.deal(), self.deck.deal()])
        self.customer = Player([self.deck.deal(), self.deck.deal()])
    
    def play(self):
        '''Play the game'''
        
        print("Customer:\n", self.customer)
        print("Dealer: \n", self.dealer) 
        
        # Ask customer to draw cards until they decide to stop
        while True: 
            user_input = input("Do you want to draw a card? [y/n]:") 
            if user_input in ("Y", "y"):
                self.customer.draw_card(self.deck) 
                 # Dealer may or may not pick a card
                self.dealer.draw_card(self.deck)
                print("Customer:\n", self.customer) 
                print("Dealer:\n", self.dealer) 
            else:
                break
            
            if self.customer.score >= 21:
                break
        
        # Last chance for dealer to draw a card
        self.dealer.draw_card(self.deck)   
        
        print("\n\n~~~~~~GAME OVER~~~~~~\n")
        print("Customer:\n", self.customer) 
        print("Dealer:\n", self.dealer)  
        # After card drawing is over, check who won
        if self.customer.score > 21: 
            print("Dealer won, you lost :(")
        elif self.dealer.score > 21:
            print("Dealer lost -- you win!")
        elif self.customer.score > self.dealer.score:
            print("You win!")
        else:
            print("Tie")

In [29]:
p = TwentyOne()

In [30]:
p.play()

Customer:
 	Cards: 3 of clubs, 2 of diamonds
	Best score: 5
Dealer: 
 Visible card: 5 of spades
Do you want to draw a card? [y/n]:y
Customer:
 	Cards: 3 of clubs, 2 of diamonds, queen of spades
	Best score: 15
Dealer:
 Visible card: 5 of spades
Do you want to draw a card? [y/n]:y
Customer:
 	Cards: 3 of clubs, 2 of diamonds, queen of spades, jack of diamonds
	Best score: 25
Dealer:
 Visible card: 5 of spades


~~~~~~GAME OVER~~~~~~

Customer:
 	Cards: 3 of clubs, 2 of diamonds, queen of spades, jack of diamonds
	Best score: 25
Dealer:
 Visible card: 5 of spades
Dealer won, you lost :(


# Putting it all together 

In [31]:
%run twenty_one.py

Customer:
 	Cards: 7 of spades, queen of hearts
	Best score: 17
Dealer: 
 	Cards: 6 of hearts, jack of spades
	Best score: 16
	Visible card: 6 of hearts
Do you want to draw a card? [y/n]:n


~~~~~~GAME OVER~~~~~~

Customer:
 	Cards: 7 of spades, queen of hearts
	Best score: 17
Dealer:
 	Cards: 6 of hearts, jack of spades, king of clubs
	Best score: 16
	Visible card: 6 of hearts
You win!


## Recap

Let's summarize a few things about the different classes here:
1. The <b>Deck</b> class has <b>Card</b> objects (<b>has a</b> relationship)
2. The <b>Player</b> class also has <b>Card</b> objects (<b>has a</b> relationship)
3. The <b>Dealer</b> class is a specific type of a <b>Player</b> (<b>is a</b> relationship)
4. The <b>TwentyOne</b> class has a <b>Deck</b>, <b>Player</b>, and <b>Dealer</b> (all are examples of <b>has a</b> relationships).


## Functional vs OOP approach 
So now we have written two different programs that play the same card game. One of them was written with a functional approach (Lab 4), and the other one (in this lecture) was written with an object-oriented approach. 

Here are some of the pros of the object-oriented approach

* Objects have real-life analogies -- this might help with the code's logic and readability. 
* Different classes can be designed separately and tested.
    * For example, we first designed the <b>Card</b> class and ensured it worked as we expected. Then, we designed the <b>Deck</b>  class, and made sure it worked as expected. Then, we started designing the <b>Player</b>  and so on.
* The objects classes that we've designed can be used in other programs
    * The <b>Deck</b>  class, for example, might be useful in creating a different card game. We've already implemented a way to shuffle a deck and deal cards from a deck. If another programmer needs to use a deck in a different program, they don't have to worry about all the details of a deck. All they need is information on the methods of the <b>Deck</b>  class (how to call the constructor, how to call the `shuffle()` method, etc).
* If we want to add more complexity (a few examples: a <b>Player</b> who is a novice, a <b>Player</b>  who has a specific strategy, a trick Deck), we can create more extensions of the classes that are already created.

This style of writing programs can be a bit strange at first. 

Thinking of a program as a series of functions may be more natural. We're used to thinking of functions as something that performs a procedure or performs a calculation. 

On the other hand, in OOP, there is a bit more design involved with thinking of what types of objects our program might need, what a class represents, what information it holds, and what procedures (i.e. methods) that object can perform. Then an added layer of complexity  -- thinking of how these objects can interact. As one example, the <b>Player</b> objects can take a <b>Deck</b> object and draw a card from it (see `Player.draw_card()` takes an argument that is a <b>Deck</b> object. The <b>Deck</b>  deals a card (`Deck.deal()` and the Player adds it to `self.hand`.)